In [14]:
from asteval import Interpreter
from google import genai
from google.genai import types
import re
import math
from dotenv import load_dotenv
import os

In [15]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


In [16]:
aeval = Interpreter()
aeval.symtable['math'] = math

gemini_client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
def send_message(user_input: str) -> str:
    system_message = """
    You are a versatile AI assistant.
    For each user query:
    - Math queries: Respond ONLY with: USE_CALCULATOR: <valid Python expression>
        Do NOT solve or simplify expressions yourself.
    - Coding queries: Respond ONLY with: USE_CODE: <valid Python code>
        Do NOT execute the code yourself or provide explanations unless explicitly asked. Provide runnable, well-formatted Python code snippets that directly solve the problem.
    - File operations and text editing queries: Respond ONLY with: USE_FILE: <valid Python code>
        Generate a Python script that performs the requested file manipulation or text modification (e.g., copying files, reading, writing, replacing text, formatting). 
        The script should be self-contained, executable, and clearly named variables. Do NOT execute the code yourself or provide explanations unless explicitly asked.
    - General knowledge: Give concise, factual answers without unnecessary detail or speculation.
    - Translation requests: Translate exactly as requested without explanations or commentary.
    - Multi-step reasoning: Break down reasoning steps clearly and concisely in logical order.
    Always keep your responses clear and strictly aligned to the query type. Do NOT mix response styles.
    For file and text editing, always respond with a Python script showing how to perform the task exactly as asked.
    """

    config = types.GenerateContentConfig(
        system_instruction=system_message,
        temperature=0.1
    )

    user_message = user_input.strip()

    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        config=config,
        contents=user_message,
    )

    print(response)

    return response.text

In [18]:
def eval_expression(expression: str) -> str:
    print(f"Evaluating expression: {expression}")
    print("-" * 20)
    result = aeval(expression)

    if aeval.error:
        print(f"Error evaluating expression: {aeval.error}")
        print("-" * 20)
        return f"Error: {aeval.error}"
    
    return result

def parse_response(response):
    print(f"Raw response: {response}")
    print("-" * 20)
    match = re.match(r"USE_CALCULATOR:\s*(.+)", response)
    if match:
        expression = match.group(1)
        return f"Result: {eval_expression(expression)}"
    return response


In [31]:
result = send_message("Change all the occurrences of 'foo' to 'bar' in the file 'example.txt'.")

print(parse_response(result))

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="USE_FILE: ```python\nimport os\n\nfilename = 'example.txt'\ntemp_filename = filename + '.tmp'\n\nwith open(filename, 'r') as infile, open(temp_filename, 'w') as outfile:\n    for line in infile:\n        outfile.write(line.replace('foo', 'bar'))\n\nos.replace(temp_filename, filename)\n```")], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, avg_logprobs=-0.03794512083364088, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=86, candidates_tokens_detai